In [5]:
# Reading boston housing dataset
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
import pandas as pd

boston = load_boston()
df = pd.DataFrame(boston.data, columns=boston.feature_names)
df.describe() # describe dataset overview

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000


In [7]:
# save files as csv
import os

WORK_DIRECTORY='data'
os.makedirs('{}'.format(WORK_DIRECTORY), exist_ok=True)
df.to_csv('{}/boston_housing.csv'.format(WORK_DIRECTORY), header=False, index=False)

In [12]:
# S3 prefix
bucket = 'sagemaker-bucket-sample-test'
prefix = 'sagemaker/sample'

# Import libraries
from sagemaker import get_execution_role
import boto3, sys, os
import sagemaker

sagemaker_session = sagemaker.Session()

# Get a SageMaker-compatible role used by this Notebook Instance.
role = get_execution_role()
my_region = boto3.session.Session().region_name # set the region of the instance
print("Execution role is " + role)
print("Success - the MySageMakerInstance is in the " + my_region + ".")

Execution role is arn:aws:iam::251344623468:role/service-role/AmazonSageMaker-ExecutionRole-20191017T203175
Success - the MySageMakerInstance is in the ap-northeast-1.


In [13]:
s3 = boto3.resource('s3')

try:
    if my_region == 'ap-northeast-1':
        s3.create_bucket(Bucket=bucket)
    else:
        s3.create_bucket(Bucket=bucket, CreateBucketConfiguration={'LocationConstraint': my_region})
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ', e)

S3 error:  An error occurred (IllegalLocationConstraintException) when calling the CreateBucket operation: The unspecified location constraint is incompatible for the region specific endpoint this request was sent to.


In [14]:
# send data to S3.SageMaker will take training data from s3
training_path = sagemaker_session.upload_data(path='{}/boston_housing.csv'.format(WORK_DIRECTORY), bucket=bucket, key_prefix=prefix)
s3_train_data = 's3://{}/{}/{}'.format(bucket, prefix, WORK_DIRECTORY)
print('Uploaded training data location: {}'.format(s3_train_data))

output_location = 's3://{}/{}/output'.format(bucket, prefix)
print('Training artifacts will be uploaded to: {}'.format(output_location))

Uploaded training data location: s3://sagemaker-getting-start-test/sagemaker/sample/data
Training artifacts will be uploaded to: s3://sagemaker-getting-start-test/sagemaker/sample/output


In [15]:
# We use the Estimator from the SageMaker Python SDK
from sagemaker.sklearn.estimator import SKLearn

script_path = 'scikit_learn_script.py'

# Initialise SDK
sklearn_estimator = SKLearn(
        entry_point=script_path,
        role = role,
        train_instance_type="ml.c4.xlarge",
        sagemaker_session=sagemaker_session,
        output_path=output_location
)

print("Estimator object: {}".format(sklearn_estimator))

Estimator object: <sagemaker.sklearn.estimator.SKLearn object at 0x7fc7954f9160>


In [16]:
# Run model training job
sklearn_estimator.fit({'train': training_path})

2020-01-13 06:19:35 Starting - Starting the training job...
2020-01-13 06:19:37 Starting - Launching requested ML instances......
2020-01-13 06:20:40 Starting - Preparing the instances for training...
2020-01-13 06:21:18 Downloading - Downloading input data...
2020-01-13 06:22:03 Training - Training image download completed. Training in progress.
2020-01-13 06:22:03 Uploading - Uploading generated training model2020-01-13 06:21:58,438 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2020-01-13 06:21:58,440 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-01-13 06:21:58,450 sagemaker_sklearn_container.training INFO     Invoking user training script.
2020-01-13 06:21:58,782 sagemaker-containers INFO     Module scikit_learn_script does not provide a setup.py. 
Generating setup.py
2020-01-13 06:21:58,782 sagemaker-containers INFO     Generating setup.cfg
2020-01-13 06:21:58,782 sagemaker-containers INFO     Generating MA

In [17]:
# Deploy an estimator and endpoint
from sagemaker.predictor import csv_serializer, json_deserializer
predictor = sklearn_estimator.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge", endpoint_name="sagemaker-terraform-test")

# Specify input and output formats.
predictor.content_type = 'text/csv'
predictor.serializer = csv_serializer
predictor.deserializer = json_deserializer

---------------------------------------------------------------------------!

In [18]:
# predictor.delete_endpoint()